In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import time
from tqdm import tqdm
import json
from contextlib import redirect_stdout
from autorank import autorank, plot_stats
import scikit_posthocs as sp
import io
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score
import hashlib
from collections import defaultdict
import itertools

## Load relevant results

## Perform stat tests

In [ ]:
def stat_tests(metric, signifigance_level = 0.05):

    column_names = [f'fold {i}' for i in range(10)]
    df = pd.DataFrame([sgd_data, scg_data, lfrog_data], columns=column_names)
    df = df.T
    df.columns = ['SGD', 'SCG', 'LFROG']

    map = {
        'val_wall': 'Validation Wall Time (Time to Convergence)',
        'val_epoch': 'Validation Epochs (Epochs to Convergence)', 
        'train_wall': 'Training Wall Time (Time to Convergence)',
        'train_epoch': 'Training Epochs (Epochs to Convergence)'
        }

    posthoc_results = sp.posthoc_nemenyi_friedman(df)
    plt.title(f'Signifigance Plot between best KNN and DT models based on {map[metric]}')
    sp.sign_plot(posthoc_results)
    plt.title('p-value')
    plt.show

    f = io.StringIO()
    with redirect_stdout(f):
        results = autorank(df, alpha=signifigance_level, verbose=False, force_mode='nonparametric', order='ascending')
    plot_stats(results)
    plt.title(f'Critical Distance plot based on {map[metric]}')
    plt.show()

    ranking_order = results.rankdf.index.tolist()
    for k in ranking_order:
        print(f"{k}: {out[metric+'_avg']} ± {out[metric+'_std']} \t\t params: {k}")